<a href="https://colab.research.google.com/github/aannsshh/Resume-Parser/blob/main/Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==3.3.1
# !python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Successfully uninstalled thinc-8.1.9
  Attempting uninstall: spacy
    Found existing 

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 16), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), 5.33 MiB | 7.03 MiB/s, done.


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
!nvidia-smi

Sun May  7 14:33:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

MessageError: ignored

In [ ]:
cv_data=json.load(open('/content/sample.json','r'))

In [ ]:
len(cv_data)

300

In [ ]:
!pip install spacy-transformers==1.1.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-05-07 14:35:48.757413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
nlp = spacy.blank('en')

In [ ]:
def get_spacy_doc(file,data):
    nlp = spacy.blank('en')
    db=DocBin()

    for text,annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot=annot['entities']

        ents=[]
        entity_indices=[]

        for start,end,label in annot:
            skip_entity=False
            for idx in range(start,end):
                if idx in entity_indices:
                    skip_entity=True
                    break
            if skip_entity==True:
                continue

            entity_indices=entity_indices+list(range(start,end))

            try:
                span=doc.char_span(start,end,label=label,alignment_mode='contract')
            except:
                continue

            if span is None:
                err_data=str([start,end])+"   "+str(text)+"\n"
                file.write(err_data)

            else:
                ents.append(span)

        try:
            doc.ents=ents
            db.add(doc)
        except:
            pass
    return db

In [ ]:
import re


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [ ]:
# len(train),len(test)
# from sklearn.model_selection import train_test_split
# train,test = train_test_split(cv,test_size=0.3)

from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.2)

In [ ]:
import spacy

In [ ]:
file =open('error.txt','w')

db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:01<00:00, 52.51it/s]


In [ ]:
# skweak-0.3.3
!pip install skweak
import skweak

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for skweak: filename=skweak-0.3.3-py3-none-any.whl size=49134 sha256=b73c1001347892df7fc4d99feb14f249e1811a32757187d23cfab2be95ab5cb0
  Stored in directory: /root/.cache/pip/wheels/1f/3f/f0/ec2ce0beb763bc547c8e4de2aa66bdc4a46a497e5d2fc7f839
Successfully built skweak


In [ ]:
new_docs = []
for doc in skweak.utils.docbin_reader("/content/train_data.spacy",spacy_model_name='en_core_web_sm'):
    new_docs.append(doc)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
doc_examples = []
for i,j in enumerate(new_docs):
    spans_ex = [(ent.start_char,ent.end_char,ent.label_) for ent in new_docs[i].ents]
    doc_examples.append([new_docs[i].text,{"entities":spans_ex}])

In [ ]:
doc_examples = trim_entity_spans(doc_examples)


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(doc_examples,test_size=0.2)

In [ ]:
train

[["Ashish Indoriya Sr. Systems Engineer at Infosys Limited  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Ashish- Indoriya/84f99c99ebe940be  • Master of Computer Application (MCA) from Bhilai Institute of Technology, Durg, 2014. • Having 3.3 years of Experience on Software Development at Infosys limited. • Extensive working experience on Java, Spring, Hibernate and SQL • Knowledge of design patterns such as Singleton, Factory, Façade, Observer and MVC. • Knowledge of Front-end web development using JavaScript, JQuery, CSS &amp; HTML. • Having knowledge of Oracle SQL Database. • Reliable as a fully contributing, responsible and accountable member of task/ project teams with highly honed creative, logical and analytical approach. • Automated some of HRMS processes like Hiring, transfer, termination to help speed up the QA process. • Hands on knowledge of C, C++ including advanced concepts such as pointers and Dynamic Memory Management. • Learning Hadoop and Big data analysis us

In [ ]:
file =open('error.txt','w')

db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 48/48 [00:00<00:00, 87.10it/s]


# Model1: ROBERTA

In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --gpu-id 0 --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy


2023-05-07 14:36:28.452070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-07 14:36:39,002] [INFO] Set up nlp object from config
[2023-05-07 14:36:39,024] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-07 14:36:39,029] [INFO] Created vocabulary
[2023-05-07 14:36:39,032] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 634kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 12.7MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 10.9MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 22.9MB/s]
Downloading: 100% 478M/478M [00:04<00:00, 116MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer

In [ ]:
# Saving the model
# nlp.to_disk('/content/drive/MyDrive/1. Resume_Parser/model-last')

NameError: ignored

In [ ]:
#Loading Model
# roberta_model = spacy.load('/content/drive/MyDrive/1. Resume_Parser/model-last')

# Model2: DistilBERT

In [ ]:
from transformers import DistilBertConfig, DistilBertModel

# Initializing a DistilBERT configuration
configuration = DistilBertConfig()

# Initializing a model (with random weights) from the configuration
model = DistilBertModel(configuration)

# Accessing the model configuration
configuration = model.config

#Model3: GPT2

In [ ]:
from transformers import GPT2Config, GPT2Model

# Initializing a GPT2 configuration
configuration = GPT2Config()

# Initializing a model (with random weights) from the configuration
model = GPT2Model(configuration)

# Accessing the model configuration
configuration = model.config

# Model4: BERT

In [ ]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install spacy==3.3.1
# !pip install thinc==8.0.17

In [ ]:
# !python -m spacy download en_core_web_lg

In [ ]:
# !pip uninstall thinc
# !pip install torch==1.1.0
# !pip install spacy-pytorch-transformers[cuda100]==0.2.0


In [ ]:
#nlp= spacy.load("en_core_web_lg")#path model best

OSError: ignored

In [ ]:
import spacy
import spacy_transformers
nlp=spacy.load("/content/output/model-best")

In [ ]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    5.3.0
cata

In [ ]:
# doc=nlp('my name is lauda lausun i have 8 years of experience I work in microsoft')

In [ ]:
for ent in doc.ents:
  print(ent.text,"  ->>>>",ent.label_)

Nikhileshkumar Ikhar   ->>>> Name
Product development engineer   ->>>> Designation
7+ years   ->>>> Years of Experience
M.Tech. in IT   ->>>> Degree
Bengaluru   ->>>> Location
indeed.com/r/Nikhileshkumar-Ikhar/ cb907948c3299ef4   ->>>> Email Address
Product Development   ->>>> Designation
Aggrigator   ->>>> Companies worked at
Bengaluru   ->>>> Location
Aggrigator   ->>>> Companies worked at
Software Engineer   ->>>> Designation
Cisco   ->>>> Companies worked at
Bengaluru   ->>>> Location
2012   ->>>> Graduation Year
Cisco   ->>>> Companies worked at
Cisco   ->>>> Companies worked at
2012   ->>>> Graduation Year
2012   ->>>> Graduation Year
System Engineer   ->>>> Designation
Infosys   ->>>> Companies worked at
M.Tech. in IT   ->>>> Degree
VIT   ->>>> College Name
IIIT   ->>>> College Name
Bengaluru   ->>>> Location
2012   ->>>> Graduation Year
Django (6 years), Java (3 years), MongoDB (3 years), MySQL (3 years), Python (6 years)   ->>>> Skills
Python, Django, Celery, Java, AngularJS, 

In [ ]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys, fitz #siz recalled variable provide fitz information extraction

In [ ]:
fname = '/content/Ansh Bhargava.pdf'
doc = fitz.open(fname)

In [ ]:
doc

Document('/content/Ansh Bhargava.pdf')

In [ ]:
text = " "
for page in doc:
  text = text +str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text

"Ansh Bhargava  \n(Assiduous, Communal, tenacity) \nPhone: +91 9598638561 \nEmail:  anshbh02@gmail.com  \nLinkedIn  : http://www.linkedin.com/in/ansh-bhargava-291258227/ \n \n \nTo secure a demanding position in a respectable organization to augment my learnings, knowledge, and skills. \nSecure a reliable career opportunity to wholly utilize my training and skills, while making a considerable \ncontribution to the company's victory. Pursuing an entry-level place to begin my career in a high-level professional \nenvironment. To secure employment with a reputable company, where I can maximize my skills and business \nstudies background. Examining a challenging role in a reputable organization to employ my technical, database, \nand management skills for the transition of the organization as well as to enrich my knowledge about new and \nemerging trends in the IT sector. \n  \n \nSubject Electives  \nDatabase management ,organizational Behavior, operational research, \nSkills \nCoding on 

In [ ]:
text = ' '.join(text.split())

In [ ]:
text

"Ansh Bhargava (Assiduous, Communal, tenacity) Phone: +91 9598638561 Email: anshbh02@gmail.com LinkedIn : http://www.linkedin.com/in/ansh-bhargava-291258227/ To secure a demanding position in a respectable organization to augment my learnings, knowledge, and skills. Secure a reliable career opportunity to wholly utilize my training and skills, while making a considerable contribution to the company's victory. Pursuing an entry-level place to begin my career in a high-level professional environment. To secure employment with a reputable company, where I can maximize my skills and business studies background. Examining a challenging role in a reputable organization to employ my technical, database, and management skills for the transition of the organization as well as to enrich my knowledge about new and emerging trends in the IT sector. Subject Electives Database management ,organizational Behavior, operational research, Skills Coding on 450 DSA- https://auth.geeksforgeeks.org/user/ans

In [ ]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text,"  ->>>>",ent.label_)

Ansh Bhargava   ->>>> Name
anshbh02@gmail.com   ->>>> Email Address
2018   ->>>> Graduation Year
2022   ->>>> Graduation Year
B. Tech in Computer Science   ->>>> Degree
Engineering Jaypee Institute Of Information Technology   ->>>> College Name
